In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn
!pip3 install joblib

You should consider upgrading via the 'c:\users\alexd\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\alexd\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\alexd\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\alexd\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

# Apartado 5

En resumen, el método elegido es el de ``SVM`` y la técnica de escalado es ``StandardScaler``. Estos serán por lo tanto los métodos que utilicemos para realizar la evalución del rendimiento futuro, como se muestra en el siguiente código:

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

# Carga los datos de entrenamiento en un DataFrame
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Selecciona solo las columnas numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Asegúrate de que 'energy' no esté en las variables numéricas, ya que será tu variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Crear el escalador
scaler = StandardScaler()

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
X_train_scaled = scaler.fit_transform(X_train)

# Transformar las características del conjunto de prueba usando el mismo escalador
X_test_scaled = scaler.transform(X_test)

# Dividir los datos de entrenamiento en un conjunto de entrenamiento más pequeño y un conjunto de validación
X_train_inner, X_val, y_train_inner, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=100472166)

# Definir el modelo SVM con los hiperparámetros seleccionados
best_model = SVR(C=1000.0, shrinking=True, degree=2, gamma='auto')

# Ajustar el modelo SVM en el conjunto de entrenamiento más pequeño
best_model.fit(X_train_inner, y_train_inner)

# Hacer predicciones en el conjunto de validación
y_val_pred = best_model.predict(X_val)

# Calcular el error cuadrático medio (RMSE) en el conjunto de validación
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
print(f"MSE en el conjunto de validación: {mse}")
print(f"RMSE en el conjunto de validación: {rmse}")

MSE en el conjunto de validación: 113942.31314937165
RMSE en el conjunto de validación: 337.5534226598386


El ``MSE`` obtenido es de 113940. Como era de esperar, al entrenarlo con todos los datos disponibles el error es significativamente menor que el resultado del ajuste de hiperparámetros (126242). El `` RMSE `` en cambio es de 337. Todos estos valores son por lo tanto los más bajos que hemos obtenido hasta ahora, pero únicamente con estas cifras no podemos determinar si nuestro modelo es bueno o no. Para ello nos guardamos el modelo entrenado:

In [4]:
from joblib import dump

# Guardar el modelo en un archivo
with open('modelo_final.pkl', 'wb') as file:
    dump(best_model, file)

Ahora utilizaremos este modelo para realizar predicciones con los datos de la competición y las guardaremos en el fichero `` predicciones.csv ``:

In [5]:
from joblib import load

# Carga los datos de competición en un DataFrame
competition_data = pd.read_csv('wind_comp.csv.gz', compression="gzip")

# Preprocesamiento: seleccionar variables numéricas
numerical_vars_competition = competition_data.select_dtypes(include=['int64', 'float64']).columns

# Escalado de los datos de la competición
X_competition_scaled = scaler.transform(competition_data[numerical_vars_competition])

# Cargar el modelo final
with open('modelo_final.pkl', 'rb') as file:
    loaded_model = load(file)

# Hacer predicciones en el conjunto de datos de la competición
competition_predictions = loaded_model.predict(X_competition_scaled)

# Guardar las predicciones en un archivo CSV
pd.DataFrame(competition_predictions).to_csv('predicciones.csv', index=False, header=False)